In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


data = pd.read_csv("datAY.csv")

predictors = ['A'] + [f'x_{i}' for i in range(1, 59)]
X = data[predictors]
y = data['Y']


model = LinearRegression()
model.fit(X, y)


y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
print("Training MSE:", mse)


X1 = X.copy()
X1['A'] = 1
X0 = X.copy()
X0['A'] = 0

Q1 = model.predict(X1)
Q0 = model.predict(X0)


estimated_ate = (Q1 - Q0).mean()
print("Estimated ATE:", estimated_ate)


ValueError: could not convert string to float: 'C'

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


data = pd.read_csv("datAY.csv", index_col=0)


predictors = ['A', 'x_1', 'x_3', 'x_4']
X = data[predictors]
y = data['Y']




model = LinearRegression()
model.fit(X, y)


y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
print("Training MSE:", mse)


X_treated = X.copy()
X_control = X.copy()
X_treated['A'] = 1   # All observations as treated
X_control['A'] = 0   # All observations as control

Q1 = model.predict(X_treated)
Q0 = model.predict(X_control)


estimated_ate = (Q1 - Q0).mean()
estimated_ate_std = (Q1 - Q0).std()
print("Estimated ATE:", estimated_ate)
print("Estimated ATE Standard Deviation:", estimated_ate_std)
print("average ate - 3 * std: ", estimated_ate - 3 * estimated_ate_std)


Training MSE: 83333.01351087335
Estimated ATE: -521.9398861268523
Estimated ATE Standard Deviation: 2.269708454168276e-13
average ate - 3 * std:  -521.939886126853


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression



data = pd.read_csv("datAY.csv", index_col=0)


predictors = ['x_1', 'x_3', 'x_4']
X_ps = data[predictors]
A = data['A']


propensity_model = LogisticRegression(solver='liblinear')
propensity_model.fit(X_ps, A)


propensity_scores = propensity_model.predict_proba(X_ps)[:, 1]


print("Propensity scores:", propensity_scores[:10])

Propensity scores: [0.43158542 0.34286201 0.34286201 0.40859634 0.56254877 0.68255377
 0.51269955 0.38175895 0.41328815 0.36859523]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


data = pd.read_csv("datAY.csv", index_col=0)


predictors = ['x_1', 'x_3', 'x_4']
X_ps = data[predictors]
A = data['A']
Y = data['Y']


propensity_model = LogisticRegression(solver='liblinear')
propensity_model.fit(X_ps, A)

propensity_scores = propensity_model.predict_proba(X_ps)[:, 1]


eps = 1e-6
propensity_scores = np.clip(propensity_scores, eps, 1 - eps)

IPTW = np.mean((Y * A / propensity_scores) - (Y * (1 - A) / (1 - propensity_scores)))
IPTW_std = np.std((Y * A / propensity_scores) - (Y * (1 - A) / (1 - propensity_scores)))
print("IPTW estimator:", IPTW)
print("IPTW estimator standard deviation:", IPTW_std)
print("IPTW estimator - 3 * std: ", IPTW - 3 * IPTW_std)

IPTW estimator: -538.6720042701767
IPTW estimator standard deviation: 6111.278774651677
IPTW estimator - 3 * std:  -18872.50832822521


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


data = pd.read_csv("datAY.csv", index_col=0)


predictors = ['x_1', 'x_3', 'x_4']
X_ps = data[predictors]
A = data['A']
Y = data['Y']


propensity_model = LogisticRegression(solver='liblinear')
propensity_model.fit(X_ps, A)
propensity_scores = propensity_model.predict_proba(X_ps)[:, 1]


eps = 1e-6
propensity_scores = np.clip(propensity_scores, eps, 1 - eps)



treated_numerator = np.sum(Y * A / propensity_scores)
treated_denominator = np.sum(A / propensity_scores)
treated_mean = treated_numerator / treated_denominator


control_numerator = np.sum(Y * (1 - A) / (1 - propensity_scores))
control_denominator = np.sum((1 - A) / (1 - propensity_scores))
control_mean = control_numerator / control_denominator


tau_h_iptw = treated_mean - control_mean

print("Stabilized h-IPTW estimator:", tau_h_iptw)
std = np.sqrt(np.var(Y * A / propensity_scores) / np.sum(A / propensity_scores) + np.var(Y * (1 - A) / (1 - propensity_scores) / np.sum((1 - A) / (1 - propensity_scores))))
print("Stabilized h-IPTW estimator standard deviation:", std)
print("Stabilized h-IPTW estimator - 3 * std: ", tau_h_iptw - 3 * std)


Stabilized h-IPTW estimator: -520.3033329316172
Stabilized h-IPTW estimator standard deviation: 50.61841346786772
Stabilized h-IPTW estimator - 3 * std:  -672.1585733352204


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

data = pd.read_csv("datAY.csv", index_col=0)

def compute_h_iptw(df):
    predictors = ['x_1', 'x_3', 'x_4']
    X_ps = df[predictors]
    A = df['A']
    Y = df['Y']
    
    propensity_model = LogisticRegression(solver='liblinear')
    propensity_model.fit(X_ps, A)
    propensity_scores = propensity_model.predict_proba(X_ps)[:, 1]
    
    eps = 1e-6
    propensity_scores = np.clip(propensity_scores, eps, 1 - eps)
    
    treated_numerator = np.sum(Y * A / propensity_scores)
    treated_denominator = np.sum(A / propensity_scores)
    treated_mean = treated_numerator / treated_denominator

    control_numerator = np.sum(Y * (1 - A) / (1 - propensity_scores))
    control_denominator = np.sum((1 - A) / (1 - propensity_scores))
    control_mean = control_numerator / control_denominator
    
    return treated_mean - control_mean

tau_h_iptw = compute_h_iptw(data)
print("h-IPTW estimator:", tau_h_iptw)

B = 1000
bootstrap_estimates = []
n = data.shape[0]

for b in range(B):
    bootstrap_sample = data.sample(n, replace=True)
    estimate = compute_h_iptw(bootstrap_sample)
    bootstrap_estimates.append(estimate)


std_est = np.std(bootstrap_estimates, ddof=1)
print("Standard deviation of the h-IPTW estimator:", std_est)
print("h-IPTW estimator - 3 * std: ", tau_h_iptw - 3 * std_est)


h-IPTW estimator: -520.3033329316172
Standard deviation of the h-IPTW estimator: 8.80431767382781


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


data = pd.read_csv("datAY.csv", index_col=0)

predictors = ['x_1', 'x_3', 'x_4']
X_ps = data[predictors]
A = data['A'].values
Y = data['Y'].values


propensity_model = LogisticRegression(solver='liblinear')
propensity_model.fit(X_ps, A)
g_hat = propensity_model.predict_proba(X_ps)[:, 1]


eps = 1e-6
g_hat = np.clip(g_hat, eps, 1 - eps)


W1 = A / g_hat
W0 = (1 - A) / (1 - g_hat)

S1 = np.sum(W1)
S0 = np.sum(W0)


mu1 = np.sum(W1 * Y) / S1
mu0 = np.sum(W0 * Y) / S0

tau_hat = mu1 - mu0
print("h-IPTW estimator:", tau_hat)

# Approximate influence functions:
IF1 = (W1 / S1) * (Y - mu1)
IF0 = (W0 / S0) * (Y - mu0)
IF_tau = IF1 - IF0  # Influence function for tau_hat

# Sandwich variance estimator:
n = len(Y)
var_tau = np.var(IF_tau, ddof=1) / n
se_tau = np.sqrt(var_tau)
print("Sandwich variance-based SE for h-IPTW:", se_tau)


h-IPTW estimator: -520.3033329316172
Sandwich variance-based SE for h-IPTW: 0.0025553243214472003


In [ ]:
import doubleml as dml

datAY = pd.read_csv("datAY.csv")
data = datAY[['x_1', 'x_3', 'x_4', 'A', 'Y']]




In [ ]:
import pandas as pd
from doubleml import DoubleMLData, DoubleMLPLR, DoubleMLIRM
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from abess.linear import LinearRegression
from sklearn.model_selection import KFold

data = pd.read_csv("new_datAY.csv", index_col=0)

x_cols = ['x_1', 'x_3', 'x_4']

dml_data = DoubleMLData(data, y_col='Y', d_cols='A', x_cols=x_cols)

# ml_g = LogisticRegressionCV(cv=5)
ml_g = LinearRegression(cv=5)
ml_m = LassoCV(cv=5)

kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_g, ml_m)

dml_plr.fit()

ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)


ATE estimate: [-585.39663319]
ATE robust (sandwich) standard error: [14.73444534]


In [56]:
# df = dgp()
from xgboost import XGBRegressor, XGBClassifier
aipw_obj_1 = DoubleMLIRM(dml_data, ml_g = LinearRegression(cv=5),
                      ml_m = LogisticRegressionCV(cv=5), n_folds=5)
aipw_obj_1.fit()

aipw_est_1 = aipw_obj_1.summary
print(aipw_est_1)


         coef   std err          t  P>|t|       2.5 %      97.5 %
A -521.405467  8.767086 -59.473061    0.0 -538.588641 -504.222294


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor, GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

kernel = 1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1, normalize_y=True)
gp_c = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=1)

obj_1 = DoubleMLIRM(dml_data, ml_g = gp, ml_m = gp_c, n_folds=5)

obj_1.fit()

est_1 = obj_1.summary
print(est_1)

/home/choudhury/miniconda3/envs/post_env/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


KeyboardInterrupt: 

In [115]:
import pandas as pd
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor, MLPClassifier


ml_m = MLPRegressor(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42)


ml_g = MLPClassifier(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42)


kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_m, ml_g, n_folds=10)


dml_plr.fit()


ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)

est2 = dml_plr.summary
print(est2)

ATE estimate: [-586.76826293]
ATE robust (sandwich) standard error: [16.68379162]
         coef    std err          t          P>|t|       2.5 %      97.5 %
A -586.768263  16.683792 -35.169959  5.759041e-271 -619.467894 -554.068632


In [ ]:
1, 3, 4, 6, 9, 10, 12

In [202]:

data = pd.read_csv("new_datAY_2.csv", index_col=0)

x_cols = ['x_1', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_22', 
          'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 
          'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58']

# x_cols = ['x_1', 'x_3', 'x_4', 'x_6', 'x_9', 'x_10', 'x_12']

dml_data = DoubleMLData(data, y_col='Y', d_cols='A', x_cols=x_cols)

ml_m = MLPRegressor(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', max_iter=500, random_state=42)


ml_g = MLPClassifier(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', max_iter=500, random_state=42)


kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_m, ml_g, n_folds=10)


dml_plr.fit()


ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)

est2 = dml_plr.summary
print(est2)

ATE estimate: [-440.89621486]
ATE robust (sandwich) standard error: [12.568061]
         coef    std err          t          P>|t|       2.5 %      97.5 %
A -440.896215  12.568061 -35.080687  1.328214e-269 -465.529162 -416.263268


In [204]:
# dml_plr.sensitvity_analysis()
dml_plr.sensitivity_plot()

ValueError: Apply sensitivity_analysis() to include senario in sensitivity_plot. 

In [ ]:
# data = pd.read_csv("datAY.csv", index_col=0)
data = pd.read_csv("more_covariates_balanced.csv", index_col=0)

x_cols = ['x_1', 'x_3', 'x_4', 'x_6', 'x_9', 'x_10', 'x_12']

dml_data = DoubleMLData(data, y_col='Y', d_cols='A', x_cols=x_cols)

# ml_g = LogisticRegressionCV(cv=5)
ml_g = LinearRegression(cv=5)
ml_m = LassoCV(cv=5)

kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_g, ml_m)

dml_plr.fit()


ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)

ATE estimate: [-1004.61863032]
ATE robust (sandwich) standard error: [45.00125123]


In [ ]:
data = pd.read_csv("datAY.csv", index_col=0)


predictors = ['x_1', 'x_3', 'x_4', 'x_6', 'x_9', 'x_10']
X_ps = data[predictors]
A = data['A'].values
Y = data['Y'].values

propensity_model = LogisticRegression(solver='liblinear')
propensity_model.fit(X_ps, A)
g_hat = propensity_model.predict_proba(X_ps)[:, 1]


eps = 1e-6
g_hat = np.clip(g_hat, eps, 1 - eps)


W1 = A / g_hat
W0 = (1 - A) / (1 - g_hat)

S1 = np.sum(W1)
S0 = np.sum(W0)

mu1 = np.sum(W1 * Y) / S1
mu0 = np.sum(W0 * Y) / S0

tau_hat = mu1 - mu0
print("h-IPTW estimator:", tau_hat)

IF1 = (W1 / S1) * (Y - mu1)
IF0 = (W0 / S0) * (Y - mu0)
IF_tau = IF1 - IF0 

n = len(Y)
var_tau = np.var(IF_tau, ddof=1) / n
se_tau = np.sqrt(var_tau)
print("Sandwich variance-based SE for h-IPTW:", se_tau)

In [ ]:
data = pd.read_csv("more_covariates_balanced.csv", index_col=0)

# "A ~ x_1 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18 + x_19 + x_20 
# + x_21 + x_23 + x_25 + x_26 + x_27 + x_28 + x_29 + x_30 + x_31 + 
# x_32 + x_33 + x_34 + x_35 + x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 
# + x_44 + x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 + x_54 + x_55 + x_56 + x_57 + x_58"


categorical_cols = ['x_11', 'x_14', 'x_21', 'x_24']


data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
print(data_encoded.columns.to_list())
x_cols = ['x_1', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 
          'x_19', 'x_20', 'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 
          'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58',
          'x_11_1', 'x_11_2', 'x_11_3', 'x_14_1', 'x_14_2', 'x_14_3', 'x_14_4', 'x_14_5', 'x_21_B', 'x_21_C', 'x_21_D', 'x_21_E', 'x_21_F', 'x_21_G', 'x_21_H', 'x_21_I', 'x_21_J', 'x_21_K', 'x_21_L', 'x_21_M', 'x_21_N', 'x_21_O', 'x_21_P', 'x_24_B', 'x_24_C', 'x_24_D', 'x_24_E']
dml_data = DoubleMLData(data_encoded, y_col='Y', d_cols='A', x_cols=x_cols)

# ml_g = LogisticRegressionCV(cv=5)
ml_g = LinearRegression(cv=5)
ml_m = LassoCV(cv=5)

kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_g, ml_m)

dml_plr.fit()


ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)

['x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_22', 'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'A', 'Y', 'prop_score', 'x_11_1', 'x_11_2', 'x_11_3', 'x_14_1', 'x_14_2', 'x_14_3', 'x_14_4', 'x_14_5', 'x_21_B', 'x_21_C', 'x_21_D', 'x_21_E', 'x_21_F', 'x_21_G', 'x_21_H', 'x_21_I', 'x_21_J', 'x_21_K', 'x_21_L', 'x_21_M', 'x_21_N', 'x_21_O', 'x_21_P', 'x_24_B', 'x_24_C', 'x_24_D', 'x_24_E']
ATE estimate: [-498.4131974]
ATE robust (sandwich) standard error: [10.75614945]


In [ ]:
np.random.seed(42)
data = pd.read_csv("more_covariates_balanced.csv", index_col=0)

# "A ~ x_1 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18 + x_19 + x_20 
# + x_21 + x_23 + x_25 + x_26 + x_27 + x_28 + x_29 + x_30 + x_31 + 
# x_32 + x_33 + x_34 + x_35 + x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 
# + x_44 + x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 + x_54 + x_55 + x_56 + x_57 + x_58"


categorical_cols = ['x_11', 'x_14', 'x_21', 'x_24']


data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
# print(data_encoded.columns.to_list())
x_cols = ['x_1', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 
          'x_19', 'x_20', 'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 
          'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58',
          'x_11_1', 'x_11_2', 'x_11_3', 'x_14_1', 'x_14_2', 'x_14_3', 'x_14_4', 'x_14_5', 'x_21_B', 'x_21_C', 'x_21_D', 'x_21_E', 'x_21_F', 'x_21_G', 'x_21_H', 'x_21_I', 'x_21_J', 'x_21_K', 'x_21_L', 'x_21_M', 'x_21_N', 'x_21_O', 'x_21_P', 'x_24_B', 'x_24_C', 'x_24_D', 'x_24_E']
dml_data = DoubleMLData(data_encoded, y_col='Y', d_cols='A', x_cols=x_cols)

ml_m = MLPRegressor(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42, early_stopping=True, alpha=0.01)


ml_g = MLPClassifier(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42, early_stopping=True, alpha=0.01)


kf = KFold(n_splits=5, shuffle=True, random_state=42)


dml_plr = DoubleMLPLR(dml_data, ml_m, ml_g, n_folds=5)


dml_plr.fit()


ate_estimate = dml_plr.coef
ate_se = dml_plr.se

print("ATE estimate:", ate_estimate)
print("ATE robust (sandwich) standard error:", ate_se)

est2 = dml_plr.summary
print(est2)

ATE estimate: [-508.1181795]
ATE robust (sandwich) standard error: [23.07124979]
         coef   std err          t          P>|t|       2.5 %      97.5 %
A -508.118179  23.07125 -22.023869  1.701034e-107 -553.336998 -462.899361


In [175]:
from xgboost import XGBRegressor, XGBClassifier
aipw_obj_1 = DoubleMLIRM(dml_data, ml_g = XGBRegressor(),
                      ml_m = XGBClassifier(), n_folds=5)
aipw_obj_1.fit()

aipw_est_1 = aipw_obj_1.summary
print(aipw_est_1)

         coef   std err           t  P>|t|       2.5 %      97.5 %
A -499.287369  1.558794 -320.303726    0.0 -502.342548 -496.232189


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

data = pd.read_csv("more_covariates_balanced.csv", index_col=0)


categorical_cols = ['x_11', 'x_14', 'x_21', 'x_24']


data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
def compute_h_iptw(df):
    predictors = ['x_1', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 
          'x_19', 'x_20', 'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 
          'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58',
          'x_11_1', 'x_11_2', 'x_11_3', 'x_14_1', 'x_14_2', 'x_14_3', 'x_14_4', 'x_14_5', 'x_21_B', 'x_21_C', 'x_21_D', 'x_21_E', 'x_21_F', 'x_21_G', 'x_21_H', 'x_21_I', 'x_21_J', 'x_21_K', 'x_21_L', 'x_21_M', 'x_21_N', 'x_21_O', 'x_21_P', 'x_24_B', 'x_24_C', 'x_24_D', 'x_24_E']
    X_ps = df[predictors]
    A = df['A']
    Y = df['Y']
    
    propensity_model = LogisticRegression(solver='liblinear')
    propensity_model.fit(X_ps, A)
    propensity_scores = propensity_model.predict_proba(X_ps)[:, 1]
    
    eps = 1e-6
    propensity_scores = np.clip(propensity_scores, eps, 1 - eps)
    
    treated_numerator = np.sum(Y * A / propensity_scores)
    treated_denominator = np.sum(A / propensity_scores)
    treated_mean = treated_numerator / treated_denominator

    control_numerator = np.sum(Y * (1 - A) / (1 - propensity_scores))
    control_denominator = np.sum((1 - A) / (1 - propensity_scores))
    control_mean = control_numerator / control_denominator
    
    return treated_mean - control_mean

tau_h_iptw = compute_h_iptw(data_encoded)
print("h-IPTW estimator:", tau_h_iptw)

B = 1000
bootstrap_estimates = []
n = data.shape[0]

for b in range(B):
    bootstrap_sample = data_encoded.sample(n, replace=True)
    estimate = compute_h_iptw(bootstrap_sample)
    bootstrap_estimates.append(estimate)


std_est = np.std(bootstrap_estimates, ddof=1)
print("Standard deviation of the h-IPTW estimator:", std_est)
print("h-IPTW estimator - 3 * std: ", tau_h_iptw - 3 * std_est)

h-IPTW estimator: -447.30115788717376
Standard deviation of the h-IPTW estimator: 99.33813056960963
h-IPTW estimator - 3 * std:  -745.3155495960027


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoLars

np.random.seed(42)
data = pd.read_csv("more_covariates_balanced.csv", index_col=0)



categorical_cols = ['x_11', 'x_14', 'x_21', 'x_24']


data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
# print(data_encoded.columns.to_list())
x_cols = ['x_1', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_12', 'x_13', 'x_15', 'x_16', 'x_17', 'x_18', 
          'x_19', 'x_20', 'x_23', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 
          'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58',
          'x_11_1', 'x_11_2', 'x_11_3', 'x_14_1', 'x_14_2', 'x_14_3', 'x_14_4', 'x_14_5', 'x_21_B', 'x_21_C', 'x_21_D', 'x_21_E', 'x_21_F', 'x_21_G', 'x_21_H', 'x_21_I', 'x_21_J', 'x_21_K', 'x_21_L', 'x_21_M', 'x_21_N', 'x_21_O', 'x_21_P', 'x_24_B', 'x_24_C', 'x_24_D', 'x_24_E']
# dml_data = DoubleMLData(data_encoded, y_col='Y', d_cols='A', x_cols=x_cols)



ml_g = LogisticRegression(solver='liblinear', random_state=42)
# mlg = MLPClassifier(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', max_iter=500, random_state=42, early_stopping=True, alpha=0.01)
ml_g.fit(data_encoded[x_cols], data_encoded['A'])
pi_hat = ml_g.predict_proba(data_encoded[x_cols])[:, 1]


data_encoded['pi_hat'] = pi_hat


X_reg = data_encoded[['A', 'pi_hat']]
Y = data_encoded['Y']

outcome_model = LinearRegression()
outcome_model.fit(X_reg, Y)
beta_hat = outcome_model.coef_[0]
gamma_hat = outcome_model.coef_[1]

print("Estimated ATE (beta):", beta_hat)
print("Coefficient on estimated propensity (gamma):", gamma_hat)


Estimated ATE (beta): -577.7947922921327
Coefficient on estimated propensity (gamma): -809.8494440031762


In [195]:
data = pd.read_csv("new_datAY_2.csv", index_col=0)
categorical_cols = ['x_11', 'x_14', 'x_21', 'x_24']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

treated = data_encoded[data_encoded['A'] == 1]
control = data_encoded[data_encoded['A'] == 0]

model_treated = LinearRegression()
model_treated.fit(treated[x_cols], treated['Y'])

model_control = LinearRegression()
model_control.fit(control[x_cols], control['Y'])

Y1_pred = model_treated.predict(data_encoded[x_cols])
Y0_pred = model_control.predict(data_encoded[x_cols])

ate_estimate = np.mean(Y1_pred - Y0_pred)
print("Estimated ATE:", ate_estimate)

Estimated ATE: -478.4445462026051


In [ ]:
treated = data_encoded[data_encoded['A'] == 1]
control = data_encoded[data_encoded['A'] == 0]


model_treated = LassoCV()
model_treated.fit(treated[x_cols], treated['Y'])


model_control = LassoCV(cv=5)
model_control.fit(control[x_cols], control['Y'])


Y1_pred = model_treated.predict(data_encoded[x_cols])
Y0_pred = model_control.predict(data_encoded[x_cols])

ate_estimate = np.mean(Y1_pred - Y0_pred)
print("Estimated ATE:", ate_estimate)

Estimated ATE: -472.7975019844525


In [201]:
X = data_encoded[['A'] + x_cols]
y = data_encoded['Y']

model = LassoCV(cv=5)
model.fit(X, y)

X_treated = X.copy()
X_control = X.copy()
X_treated['A'] = 1
X_control['A'] = 0


Y1_pred = model.predict(X_treated)
Y0_pred = model.predict(X_control)

ate_estimate = np.mean(Y1_pred - Y0_pred)
print("Estimated ATE:", ate_estimate)

Estimated ATE: -388.4398355342925


In [ ]:
treated = data_encoded[data_encoded['A'] == 1]
control = data_encoded[data_encoded['A'] == 0]


model_treated = MLPRegressor(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42, early_stopping=True, alpha=0.01)
model_treated.fit(treated[x_cols], treated['Y'])


model_control = MLPRegressor(hidden_layer_sizes=(32, 32), activation='relu', solver='adam', max_iter=500, random_state=42, early_stopping=True, alpha=0.01)
model_control.fit(control[x_cols], control['Y'])


Y1_pred = model_treated.predict(data_encoded[x_cols])
Y0_pred = model_control.predict(data_encoded[x_cols])


ate_estimate = np.mean(Y1_pred - Y0_pred)
print("Estimated ATE:", ate_estimate)

/home/choudhury/miniconda3/envs/post_env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.



Estimated ATE: -603.4135714812794


/home/choudhury/miniconda3/envs/post_env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.

